# Tuning classifiers (MLP)

### Setup

In [1]:
%run -i 'random_state.py'
from packages import *
from clean_functions import *
from tokenizer import *
from tuners import *

General grid

In [2]:
hyper_mlp = {'neurons': [10, 25, 50, 75, 100, 150, 200], #hidden LSTM
             'lamb1': [.0, 10**-6, 5*10**-6, 10**-5, 5*10**-5, 10**-4, 5*10**-4, 10**-3, 5*10**-3], #regularization
             'lamb2': [.0, 10**-6, 5*10**-6, 10**-5, 5*10**-5, 10**-4, 5*10**-4, 10**-3, 5*10**-3],
             'score': [0], 
             'lower_ci': [0], 
             'upper_ci': [0]}

# W2V/MLP

### Getting data ready

Loading data

In [3]:
X, y=np.load('data/X_w2v.npy'),np.load('data/y_w2v.npy')

In [4]:
N=np.sum(X[:,:,:,0]!=0, axis=2)
N=np.expand_dims(N, axis=2)
N[N==0]=1
X=np.sum(X, axis=2)
X=X/N
X=X.reshape((X.shape[0],-1))

Turning y into numeric:

In [5]:
encode={'H:Arquivado': 1,'H:Ativo': 2,'H:Suspenso': 3}
decode={1:'H:Arquivado',2:'H:Ativo',3:'H:Suspenso'}

for i in range(len(y)):
    y[i]=encode[y[i]]

Splitting the dataset in train, test and validation set:

In [6]:
y=np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=2/3, random_state=random_seed)

y_train2=np.array(pd.get_dummies(y_train))
y_val2=np.array(pd.get_dummies(y_val))
y_test2=np.array(pd.get_dummies(y_test))

np.shape(X_train),np.shape(y_train)

((4514, 500), (4514,))

### Tuning classification model

Grid

In [7]:
hyper=expand_grid(hyper_mlp.copy(), random_seed=random_seed)
hyper=hyper[['neurons','lamb1','lamb2','score','lower_ci','upper_ci']]

np.shape(hyper)

(50, 6)

Search

In [8]:
hyper=tune_mlp(hyper, X_train, y_train2, X_val, y_val2)
hyper.to_csv('hyper/hyper_mlp_w2v')

100%|██████████| 50/50 [03:21<00:00,  4.04s/it]


In [9]:
hyper.iloc[np.argsort(hyper.loc[:,'score']),:].tail(20)

,neurons,lamb1,lamb2,score,lower_ci,upper_ci
10,75,0.000000,0.000001,0.902326,0.879414,0.925237
17,50,0.000050,0.000050,0.902326,0.879414,0.925237
23,100,0.000010,0.000001,0.902326,0.879414,0.925237
39,75,0.000001,0.000005,0.903876,0.881128,0.926624
38,25,0.000005,0.000010,0.903876,0.881128,0.926624
40,25,0.000010,0.000010,0.905426,0.882843,0.928010
22,150,0.000100,0.000050,0.905426,0.882843,0.928010
19,75,0.000050,0.000010,0.905426,0.882843,0.928010
32,200,0.000050,0.000001,0.905426,0.882843,0.928010
28,200,0.000100,0.000100,0.905426,0.882843,0.928010


# BERT/MLP

### Getting data ready

Loading data:

In [10]:
X, y = np.load('data/X_bert.npy'),np.load('data/y_bert.npy')

In [11]:
X=X.reshape((X.shape[0],-1))

Turning y into numeric:

In [12]:
encode={'H:Arquivado': 1,'H:Ativo': 2,'H:Suspenso': 3}
decode={1:'H:Arquivado',2:'H:Ativo',3:'H:Suspenso'}

for i in range(len(y)):
    y[i]=encode[y[i]]

Splitting the dataset in train, test and validation set:

In [13]:
y=np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=2/3, random_state=random_seed)

y_train2=np.array(pd.get_dummies(y_train))
y_val2=np.array(pd.get_dummies(y_val))
y_test2=np.array(pd.get_dummies(y_test))

np.shape(X_train),np.shape(y_train)

((4514, 3840), (4514,))

### Tuning classification model

Grid

In [14]:
hyper=expand_grid(hyper_mlp.copy(), random_seed=random_seed)
hyper=hyper[['neurons','lamb1','lamb2','score','lower_ci','upper_ci']]

np.shape(hyper)

(50, 6)

Grid Search

In [15]:
hyper=tune_mlp(hyper, X_train, y_train2, X_val, y_val2)
hyper.to_csv('hyper/hyper_mlp_bert')

100%|██████████| 50/50 [06:32<00:00,  7.85s/it]


In [16]:
hyper.iloc[np.argsort(hyper.loc[:,'score']),:].tail(20)

,neurons,lamb1,lamb2,score,lower_ci,upper_ci
14,50,0.000001,0.001000,0.906977,0.884560,0.929393
5,200,0.000100,0.000010,0.908527,0.886279,0.930775
29,50,0.000001,0.000005,0.908527,0.886279,0.930775
12,75,0.000001,0.000010,0.911628,0.889723,0.933533
38,25,0.000005,0.000010,0.911628,0.889723,0.933533
19,75,0.000050,0.000010,0.911628,0.889723,0.933533
23,100,0.000010,0.000001,0.913178,0.891448,0.934909
44,25,0.000010,0.000000,0.913178,0.891448,0.934909
46,150,0.000005,0.000010,0.913178,0.891448,0.934909
31,50,0.000050,0.000100,0.914729,0.893175,0.936282


# Doc2Vec/MLP

### Getting data ready

Loading data

In [17]:
X, y=np.load('data/X_d2v.npy'),np.load('data/y_d2v.npy')

In [18]:
X=X.reshape((X.shape[0],-1))

Turning y into numeric:

In [19]:
encode={'H:Arquivado': 1,'H:Ativo': 2,'H:Suspenso': 3}
decode={1:'H:Arquivado',2:'H:Ativo',3:'H:Suspenso'}

for i in range(len(y)):
    y[i]=encode[y[i]]

Splitting the dataset in train, test and validation set:

In [20]:
y=np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=2/3, random_state=random_seed)

y_train2=np.array(pd.get_dummies(y_train))
y_val2=np.array(pd.get_dummies(y_val))
y_test2=np.array(pd.get_dummies(y_test))

np.shape(X_train),np.shape(y_train)

((4514, 500), (4514,))

### Tuning classification model

Grid

In [21]:
hyper=expand_grid(hyper_mlp.copy(), random_seed=random_seed)
hyper=hyper[['neurons','lamb1','lamb2','score','lower_ci','upper_ci']]

np.shape(hyper)

(50, 6)

Search

In [22]:
hyper=tune_mlp(hyper, X_train, y_train2, X_val, y_val2)
hyper.to_csv('hyper/hyper_mlp_d2v')

100%|██████████| 50/50 [03:28<00:00,  4.17s/it]


In [23]:
hyper.iloc[np.argsort(hyper.loc[:,'score']),:].tail(20)

,neurons,lamb1,lamb2,score,lower_ci,upper_ci
9,10,0.000500,0.000100,0.784496,0.752764,0.816228
5,200,0.000100,0.000010,0.784496,0.752764,0.816228
47,100,0.001000,0.001000,0.784496,0.752764,0.816228
37,25,0.000100,0.000001,0.786047,0.754398,0.817696
8,75,0.000005,0.000100,0.786047,0.754398,0.817696
30,10,0.000100,0.000001,0.786047,0.754398,0.817696
40,25,0.000010,0.000010,0.786047,0.754398,0.817696
10,75,0.000000,0.000001,0.786047,0.754398,0.817696
46,150,0.000005,0.000010,0.786047,0.754398,0.817696
49,150,0.000001,0.000001,0.787597,0.756032,0.819162


# TFIDF/MLP

### Getting data ready

Loading data:

In [24]:
X, y=np.load('data/X_tfidf.npy'),np.load('data/y_tfidf.npy')

In [25]:
X=X.reshape((X.shape[0],-1))

Turning y into numeric:

In [26]:
encode={'H:Arquivado': 1,'H:Ativo': 2,'H:Suspenso': 3}
decode={1:'H:Arquivado',2:'H:Ativo',3:'H:Suspenso'}

for i in range(len(y)):
    y[i]=encode[y[i]]

Splitting the dataset in train, test and validation set:

In [27]:
y=np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=2/3, random_state=random_seed)

y_train2=np.array(pd.get_dummies(y_train))
y_val2=np.array(pd.get_dummies(y_val))
y_test2=np.array(pd.get_dummies(y_test))

np.shape(X_train),np.shape(y_train)

((4514, 20000), (4514,))

### Tuning classification model

Grid

In [28]:
hyper=expand_grid(hyper_mlp.copy(), random_seed=random_seed)
hyper=hyper[['neurons','lamb1','lamb2','score','lower_ci','upper_ci']]

np.shape(hyper)

(50, 6)

Search

In [29]:
hyper=tune_mlp(hyper, X_train, y_train2, X_val, y_val2)
hyper.to_csv('hyper/hyper_mlp_tfidf')

100%|██████████| 50/50 [30:46<00:00, 36.93s/it]


In [30]:
hyper.iloc[np.argsort(hyper.loc[:,'score']),:].tail(20)

,neurons,lamb1,lamb2,score,lower_ci,upper_ci
24,10,0.000010,0.000001,0.910078,0.888000,0.932155
31,50,0.000050,0.000100,0.911628,0.889723,0.933533
5,200,0.000100,0.000010,0.911628,0.889723,0.933533
13,200,0.000100,0.000001,0.911628,0.889723,0.933533
14,50,0.000001,0.001000,0.911628,0.889723,0.933533
0,25,0.000050,0.000500,0.911628,0.889723,0.933533
21,25,0.000050,0.000100,0.913178,0.891448,0.934909
3,150,0.000001,0.001000,0.913178,0.891448,0.934909
35,150,0.000100,0.000500,0.914729,0.893175,0.936282
17,50,0.000050,0.000050,0.914729,0.893175,0.936282
